In [15]:
import sys

sys.path.append("../")

In [27]:
from clonal_selection.clonal_selection_anti_worse_pro_elite import ClonalSelectionAntiWorseProElite
from clonal_selection.clonal_selection_cognitive import ClonalSelectionCognitive
from clonal_selection.util import get_mean_solution, get_mean_result, get_mean_history
from jmetal.problem.singleobjective.unconstrained import Rastrigin
from jmetal.util.termination_criterion import StoppingByEvaluations

from jmetal.operator import PolynomialMutation, SimpleRandomMutation

from clonal_selection.clonal_selection import ClonalSelection
import pandas as pd
import time

In [34]:
import itertools 
from pprint import pprint

# Clonal Selection

In [36]:
number_of_variables = [50, 100, 200] # 500?
population_size = [100, 200, 500]
selection_size = [2/20, 3/20, 4/20,  5/20]
mutation = ["polynomial", "random"]
mutation_probability = [1,2,5]
clone_rate = [2/20, 5/20, 10/20]
random_cells_number = [2/20,5/20,10/20]

grid = [number_of_variables, population_size, selection_size, mutation, mutation_probability, clone_rate, random_cells_number]

grid = list(itertools.product(*grid))
pprint(grid[:5])

[(50, 100, 0.1, 'polynomial', 1, 0.1, 0.1),
 (50, 100, 0.1, 'polynomial', 1, 0.1, 0.25),
 (50, 100, 0.1, 'polynomial', 1, 0.1, 0.5),
 (50, 100, 0.1, 'polynomial', 1, 0.25, 0.1),
 (50, 100, 0.1, 'polynomial', 1, 0.25, 0.25)]


In [ ]:
max_evaluations = 10
number_of_tries = 2

for n, ps, ss, m, mp, cr, rcn in grid:
    problem = Rastrigin(number_of_variables=n)
    results = []
    histories = []
    for i in range(number_of_tries):
        cs_algo = ClonalSelection(
            problem=problem,
            population_size=ps,
            selection_size=int(ss * ps),
            clone_rate = int(cr * ps), 
            random_cells_number = int(rcn * ps),
            mutation=PolynomialMutation(probability = mp / problem.number_of_variables) if m == "polynomial" else SimpleRandomMutation(probability = mp / problem.number_of_variables),
            termination_criterion=StoppingByEvaluations(max_evaluations=max_evaluations),
            debug=True
        )

        cs_algo.run()
        results.append(cs_algo.get_result())
        histories.append([s.objectives[0] for s in cs_algo.history])

    print('Algorithm: ' + cs_algo.get_name())
    print('Problem: ' + problem.get_name())
    print('Solution: ' + str(get_mean_solution(results)))
    print('Fitness:  ' + str(get_mean_result(results)))
    cs_history = get_mean_history(histories)

    results = {
        "problem":"Rastrigin", 
        "number_of_variables": n, 
        "population_size": ps,
        "selection_size": ss,
        "mutation": m,
        "mutation_probability":mp, 
        "clone_rate": cr,
        "random_cells_number":rcn,
        "results": histories}
    with open(f'clonal_selection_rastrigin_{n}_{ps}_{ss}_{m}_{mp}_{cr}_{rcn}.json', 'w') as outfile:
        json.dump(results, outfile)